# 붓꽃(Iris) 품종 분류 모델

## 데이터 로드

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

In [ ]:
df = pd.DataFrame(iris.data) 
df.columns = iris.feature_names 
df['label'] = iris.target 

In [ ]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## 입력데이터(input data)와 레이블(label) 분리

In [ ]:
X = df.drop('label', axis=1).to_numpy() 
Y = df['label'].to_numpy().reshape((-1,1))

In [ ]:
X.shape

(150, 4)

In [ ]:
Y.shape

(150, 1)

## 입력 데이터 정규화

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## 훈련데이터셋과 테스트데이터셋으로 분리

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

## 데이터셋 클래스

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset 

class TensorData(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.LongTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index] 

    def __len__(self):
        return self.len

## 데이터 로더

In [ ]:
train_ds = TensorData(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)

test_ds = TensorData(X_test, Y_test)
testloader = torch.utils.data.DataLoader(test_ds, batch_size=16, shuffle=False)

In [ ]:
test_ds[0]

(tensor([ 0.3110, -0.5924,  0.5354,  0.0009]), tensor(1))

## 신경망 모델 아키텍처

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(4, 200)
        self.fc2 = nn.Linear(200, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)
        return X

## 신경망 모델 객체 생성

In [ ]:
model = Classifier()

## 손실 함수

In [ ]:
criterion = nn.CrossEntropyLoss()

## 옵티마이저를 생성하고, 생성한 모델의 파라미터를  최적화 대상으로 등록


In [ ]:
 optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-7)

## 신경망 모델 훈련(학습)

In [ ]:
epochs = 10

for epoch in range(epochs):
    print(f'### epoch {epoch+1} ###############')
    for i, data in enumerate(trainloader, 0): 
        input, target = data 
        optimizer.zero_grad()
        pred = model(input)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        print(f'{i+1} : Loss {loss}')

### epoch 1 ###############
1 : Loss 1.0966870784759521
2 : Loss 1.0687446594238281
3 : Loss 1.0175457000732422
4 : Loss 1.0116595029830933
5 : Loss 1.0091655254364014
6 : Loss 0.8980752825737
7 : Loss 0.9020828604698181
### epoch 2 ###############
1 : Loss 0.8000479936599731
2 : Loss 0.8227065801620483
3 : Loss 0.7491028904914856
4 : Loss 0.8052451014518738
5 : Loss 0.8107287287712097
6 : Loss 0.8875216245651245
7 : Loss 0.9376950860023499
### epoch 3 ###############
1 : Loss 0.8100994825363159
2 : Loss 0.7622026801109314
3 : Loss 0.7348368167877197
4 : Loss 0.6670273542404175
5 : Loss 0.8152891993522644
6 : Loss 0.682107150554657
7 : Loss 0.7522532343864441
### epoch 4 ###############
1 : Loss 0.6974712610244751
2 : Loss 0.7867059707641602
3 : Loss 0.7650312185287476
4 : Loss 0.6753073930740356
5 : Loss 0.718168318271637
6 : Loss 0.6943297386169434
7 : Loss 0.6233030557632446
### epoch 5 ###############
1 : Loss 0.7218785881996155
2 : Loss 0.6548914909362793
3 : Loss 0.75923877954483

## 신경망 모델 평가

In [ ]:
correct = 0
with torch.no_grad():
  for i, data in enumerate(test_ds):
    label = data[1].numpy()
    output = model.forward(data[0].reshape(1,-1))
    pred = output.argmax().item()
    
    if label == pred:
      correct += 1

print(f'정확도 : {correct/len(test_ds)*100:.2f}%')

정확도 : 95.56%
